In [22]:
#Importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


STEP 1: Load & Clean Data

In [23]:
df = pd.read_excel("/content/Online Retail.xlsx", sheet_name="Online Retail")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [24]:
# Remove missing CustomerID
df = df[df['CustomerID'].notnull()]

In [25]:
# Remove cancellations (InvoiceNo starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [26]:
# Create TotalAmount = Quantity * UnitPrice
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


STEP 2: RFM Calculation

In [27]:
ref_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

In [28]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (ref_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                             # Frequency
    'TotalAmount': 'sum'                                # Monetary
}).reset_index()

In [29]:
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalAmount': 'Monetary'
}, inplace=True)

STEP 3: RFM Scoring

In [30]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method="first"), 4, labels=[1,2,3,4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])

In [31]:
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].sum(axis=1).astype(int)

STEP 4: Define Segments

In [32]:
def segment_customer(r, f, m):
    if r <= 25 and f >= 10 and m >= 1000:
        return "Best Customers"
    elif f >= 8:
        return "Loyal Customers"
    elif m >= 500:
        return "Big Spenders"
    elif r >= 90 and f <= 2:
        return "Almost Lost"
    elif r >= 180:
        return "Lost Customers"
    elif r >= 90 and m < 100:
        return "Lost Cheap Customers"
    else:
        return "Others"

rfm['Segment'] = rfm.apply(lambda x: segment_customer(x['Recency'], x['Frequency'], x['Monetary']), axis=1)

STEP 5: Merge with Original Data (to create NEW dataset)

In [33]:
# Take unique customer info (CustomerID + Country, etc.)
customer_info = df.groupby('CustomerID').agg({
    'Country': 'first'
}).reset_index()

In [34]:
# Merge RFM results with customer info
final_dataset = pd.merge(customer_info, rfm, on="CustomerID", how="left")

In [38]:
print(final_dataset.head())

   CustomerID         Country  Recency  Frequency  Monetary R_Score F_Score  \
0     12346.0  United Kingdom      326          1  77183.60       1       1   
1     12347.0         Iceland        2          7   4310.00       4       4   
2     12348.0         Finland       75          4   1797.24       2       3   
3     12349.0           Italy       19          1   1757.55       3       1   
4     12350.0          Norway      310          1    334.40       1       1   

  M_Score RFM_Segment  RFM_Score       Segment  
0       4         114          6  Big Spenders  
1       4         444         12  Big Spenders  
2       4         234          9  Big Spenders  
3       4         314          8  Big Spenders  
4       2         112          4   Almost Lost  


STEP 6: Save New Dataset

In [39]:
# Save as CSV
final_dataset.to_csv("Customer_Segmentation_Dataset.csv", index=False)

In [40]:
# If in Google Colab → download the file
from google.colab import files
files.download("Customer_Segmentation_Dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>